In [53]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_default_device(device)
device

'cpu'

In [54]:
# import requests
# from pathlib import Path 

# # Download helper functions from Learn PyTorch repo (if not already downloaded)
# if Path("helper_functions.py").is_file():
#   print("helper_functions.py already exists, skipping download")
# else:
#   print("Downloading helper_functions.py")
#   # Note: you need the "raw" GitHub URL for this to work
#   request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
#   with open("helper_functions.py", "wb") as f:
#     f.write(request.content)

In [55]:
!pip3 install tqdm

In [56]:
df = pd.read_csv('baseline_data.csv')

/var/folders/cr/n_94fxl11c57rv6csplpw9dc0000gn/T/ipykernel_73611/2305619842.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('baseline_data.csv')


In [57]:
df.head()

,mrd_type,original_list_price,list_price,close_price,association_fee,tax_annual_amount,days_on_market,postal_code,rooms_total,bedrooms_total,...,garage_spaces,lot_size_acres,living_area,year_built,public_remarks,elementary_school_district,middle_or_junior_school_district,high_school_district,waterfront_yn,street_dir_prefix
0,Detached Single,189900.0,189900.0,190000.0,0.0,1102.0,3,46366,9,3,...,2.0,0.0,1886,1875,"Perfectly situated on 10 acres of land, this 3...",NaN,NaN,NaN,False,W
1,Attached Single,2992002.0,2992002.0,2749916.0,1619.0,0.0,26,60601,7,3,...,1.0,0.0,2586,0,Introducing Vista Residences Chicago's New Lux...,299,299,299,True,E
2,Attached Single,1969100.0,1969100.0,1670267.0,1064.0,0.0,2,60601,6,3,...,1.0,0.0,2804,0,Introducing Vista Residences Chicago's New Lux...,299,299,299,True,E
3,Attached Single,1164240.0,1164240.0,1085145.0,633.0,0.0,2,60601,4,1,...,1.0,0.0,1176,0,Introducing Vista Residences Chicago's New Lux...,299,299,299,True,E
4,Attached Single,1380000.0,1470670.0,1306059.0,750.0,0.0,5,60601,7,2,...,1.0,0.0,2000,0,Introducing Vista Residences Chicago's New Lux...,299,299,299,True,E


In [58]:
df.info()
df = df.drop(['street_dir_prefix'],axis=1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400918 entries, 0 to 400917
Data columns (total 22 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   mrd_type                          400918 non-null  object 
 1   original_list_price               400918 non-null  float64
 2   list_price                        400918 non-null  float64
 3   close_price                       400918 non-null  float64
 4   association_fee                   400918 non-null  float64
 5   tax_annual_amount                 400918 non-null  float64
 6   days_on_market                    400918 non-null  int64  
 7   postal_code                       400918 non-null  object 
 8   rooms_total                       400918 non-null  int64  
 9   bedrooms_total                    400918 non-null  int64  
 10  bathrooms_full                    400918 non-null  int64  
 11  bathrooms_half                    400918 non-null  i

In [59]:
df.columns

Index(['mrd_type', 'original_list_price', 'list_price', 'close_price',
       'association_fee', 'tax_annual_amount', 'days_on_market', 'postal_code',
       'rooms_total', 'bedrooms_total', 'bathrooms_full', 'bathrooms_half',
       'garage_spaces', 'lot_size_acres', 'living_area', 'year_built',
       'public_remarks', 'elementary_school_district',
       'middle_or_junior_school_district', 'high_school_district',
       'waterfront_yn'],
      dtype='object')

In [60]:
unique_elementary = df[["postal_code","elementary_school_district"]].groupby(["postal_code","elementary_school_district"]).size().reset_index().rename(columns={0:'count'})
unique_middle = df[["postal_code","middle_or_junior_school_district"]].groupby(["postal_code","middle_or_junior_school_district"]).size().reset_index().rename(columns={0:'count'})
unique_high = df[["postal_code","high_school_district"]].groupby(["postal_code","high_school_district"]).size().reset_index().rename(columns={0:'count'})

In [61]:
el_dict = dict(zip(unique_elementary.postal_code, unique_elementary.elementary_school_district))
md_dict = dict(zip(unique_middle.postal_code, unique_middle.middle_or_junior_school_district))
hs_dict = dict(zip(unique_high.postal_code, unique_high.high_school_district))

In [62]:
df.public_remarks = df.public_remarks.fillna('')
df.elementary_school_district = df.elementary_school_district.fillna(df.postal_code.map(el_dict))
df.middle_or_junior_school_district = df.middle_or_junior_school_district.fillna(df.postal_code.map(md_dict))
df.high_school_district = df.high_school_district.fillna(df.postal_code.map(hs_dict))

In [63]:
df = df.dropna(axis=0)

In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 400040 entries, 1 to 400917
Data columns (total 21 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   mrd_type                          400040 non-null  object 
 1   original_list_price               400040 non-null  float64
 2   list_price                        400040 non-null  float64
 3   close_price                       400040 non-null  float64
 4   association_fee                   400040 non-null  float64
 5   tax_annual_amount                 400040 non-null  float64
 6   days_on_market                    400040 non-null  int64  
 7   postal_code                       400040 non-null  object 
 8   rooms_total                       400040 non-null  int64  
 9   bedrooms_total                    400040 non-null  int64  
 10  bathrooms_full                    400040 non-null  int64  
 11  bathrooms_half                    400040 non-null  int64 

In [65]:
cat_cols = list(df.select_dtypes(include=['object','bool']).columns)
num_cols = list(df.select_dtypes(include=['float64','int64']).columns)

cat_cols.remove('public_remarks',)
num_cols.remove('close_price')

df_text = df[['public_remarks']].reset_index(drop='True').astype('string')
df_target = df[['close_price']].reset_index(drop='True')

df_cat = df[cat_cols].reset_index(drop='True').astype('string')

df_num = df[num_cols].reset_index(drop='True')

In [66]:
df_cat.columns

Index(['mrd_type', 'postal_code', 'elementary_school_district',
       'middle_or_junior_school_district', 'high_school_district',
       'waterfront_yn'],
      dtype='object')

In [67]:
df_num.columns

Index(['original_list_price', 'list_price', 'association_fee',
       'tax_annual_amount', 'days_on_market', 'rooms_total', 'bedrooms_total',
       'bathrooms_full', 'bathrooms_half', 'garage_spaces', 'lot_size_acres',
       'living_area', 'year_built'],
      dtype='object')

In [68]:
df_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400040 entries, 0 to 400039
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   original_list_price  400040 non-null  float64
 1   list_price           400040 non-null  float64
 2   association_fee      400040 non-null  float64
 3   tax_annual_amount    400040 non-null  float64
 4   days_on_market       400040 non-null  int64  
 5   rooms_total          400040 non-null  int64  
 6   bedrooms_total       400040 non-null  int64  
 7   bathrooms_full       400040 non-null  int64  
 8   bathrooms_half       400040 non-null  int64  
 9   garage_spaces        400040 non-null  float64
 10  lot_size_acres       400040 non-null  float64
 11  living_area          400040 non-null  int64  
 12  year_built           400040 non-null  int64  
dtypes: float64(6), int64(7)
memory usage: 39.7 MB


In [69]:
df_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400040 entries, 0 to 400039
Data columns (total 6 columns):
 #   Column                            Non-Null Count   Dtype 
---  ------                            --------------   ----- 
 0   mrd_type                          400040 non-null  string
 1   postal_code                       400040 non-null  string
 2   elementary_school_district        400040 non-null  string
 3   middle_or_junior_school_district  400040 non-null  string
 4   high_school_district              400040 non-null  string
 5   waterfront_yn                     400040 non-null  string
dtypes: string(6)
memory usage: 18.3 MB


In [70]:
df_target.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400040 entries, 0 to 400039
Data columns (total 1 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   close_price  400040 non-null  float64
dtypes: float64(1)
memory usage: 3.1 MB


In [71]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(categories='auto')
feature_arr = ohe.fit_transform(df_cat).toarray()
feature_labels = ohe.get_feature_names_out()
df_cat_encoded = pd.DataFrame(feature_arr, columns=feature_labels)
df_cat_encoded.head()

,mrd_type_Attached Single,mrd_type_Detached Single,postal_code_46075,postal_code_46168,postal_code_46324,postal_code_46360,postal_code_46373,postal_code_47804,postal_code_47847,postal_code_47862,...,high_school_district_VALM,high_school_district_WAS3,high_school_district_WATE,high_school_district_WEBS,high_school_district_WENT,high_school_district_keno,high_school_district_kenos,high_school_district_salem,waterfront_yn_False,waterfront_yn_True
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [72]:
df_features = pd.concat([df_num,df_cat_encoded], axis = 1)

In [75]:
df_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400040 entries, 0 to 400039
Columns: 2350 entries, original_list_price to waterfront_yn_True
dtypes: float64(2343), int64(7)
memory usage: 7.0 GB


In [76]:
df_text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400040 entries, 0 to 400039
Data columns (total 1 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   public_remarks  400040 non-null  string
dtypes: string(1)
memory usage: 3.1 MB


In [233]:
df_features.isnull().values.any()

False

In [260]:
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

BATCH_SIZE = 64

X_train, X_test, y_train, y_test = train_test_split(df_features, df_target, test_size=0.33)

min_max_scale = MinMaxScaler()
X_train[num_cols] = min_max_scale.fit_transform(X_train[num_cols])
X_test[num_cols] = min_max_scale.transform(X_test[num_cols])

print(X_train.isnull().values.any(),X_test.isnull().values.any())

X_train = torch.from_numpy(X_train.values).type(torch.float)
X_test = torch.from_numpy(X_test.values).type(torch.float)
y_train = torch.from_numpy(y_train.values).type(torch.float)
y_test = torch.from_numpy(y_test.values).type(torch.float)

False False


In [261]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

(torch.Size([268026, 2350]),
 torch.Size([132014, 2350]),
 torch.Size([268026, 1]),
 torch.Size([132014, 1]))

In [262]:
train_data = list(zip(X_train,y_train))
test_data = list(zip(X_test,y_test))

In [263]:
train_dataloader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=BATCH_SIZE)
test_dataloader = torch.utils.data.DataLoader(test_data, shuffle=True, batch_size=BATCH_SIZE)

In [264]:
tmp_x, tmp_y, = next(iter(train_dataloader))
tmp_x.shape,tmp_y.shape

(torch.Size([64, 2350]), torch.Size([64, 1]))

In [265]:
input_shape = tmp_x.shape[-1]
output_shape = 1

In [288]:
class RealEstatePredictor(nn.Module):
    def __init__(self, input_shape: int, hidden_units_1: int, hidden_units_2: int, output_shape: int, p: float):
        super().__init__()
        self.linear1 = nn.Linear(in_features=input_shape, out_features=hidden_units_1)
        self.linear2 = nn.Linear(in_features=hidden_units_1, out_features=hidden_units_2)
        # self.dropout1 = nn.Dropout(p=p)
        self.hidden_act = nn.ReLU()
        self.linear3 = nn.Linear(in_features=hidden_units_2, out_features=output_shape)
    
    def forward(self, x):
        x = self.linear1(x)
        x = self.linear2(x)
        # x = self.dropout1(x)
        x = self.hidden_act(x)
        x = self.linear3(x)
        return x

In [289]:
def weights_init_uniform_rule(m):
    classname = m.__class__.__name__
    # for every Linear layer in a model..
    if classname.find('Linear') != -1:
        # get the number of the inputs
        n = m.in_features
        y = 1.0/np.sqrt(n)
        m.weight.data.uniform_(-y, y)
        m.bias.data.fill_(0)

In [295]:
model_0 = RealEstatePredictor(input_shape=input_shape,
    hidden_units_1=32, # how many units in the hiden layer
    hidden_units_2=32, # how many units in the hiden layer
    output_shape=output_shape, # one for every class
    p = .5
)
model_0.apply(weights_init_uniform_rule)
model_0.to(device)

RealEstatePredictor(
  (linear1): Linear(in_features=2350, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=32, bias=True)
  (hidden_act): ReLU()
  (linear3): Linear(in_features=32, out_features=1, bias=True)
)

In [296]:
model_0.state_dict()

OrderedDict([('linear1.weight',
              tensor([[-1.5488e-02,  7.3230e-03,  1.1165e-02,  ...,  1.0597e-02,
                       -1.0037e-02,  3.5830e-03],
                      [ 9.3376e-03,  2.0170e-02,  1.4258e-03,  ...,  1.4375e-02,
                        1.5698e-02,  7.0407e-03],
                      [ 7.3945e-03,  8.5566e-03,  1.9269e-02,  ...,  8.9222e-03,
                       -9.8756e-03,  1.8247e-03],
                      ...,
                      [-1.2071e-02,  3.4852e-03,  1.9877e-02,  ...,  4.3430e-03,
                       -1.2909e-02, -1.5038e-02],
                      [-4.2998e-03, -2.8140e-03,  9.2278e-03,  ...,  1.1798e-02,
                       -6.4924e-05, -1.0255e-02],
                      [ 1.3021e-02, -6.4639e-03,  1.6056e-02,  ...,  1.5052e-02,
                       -1.4603e-02,  1.6169e-02]])),
             ('linear1.bias',
              tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    

In [297]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(params=model_0.parameters(), lr=0.01)

In [298]:
from timeit import default_timer as timer 
def print_train_time(start: float, end: float, device: torch.device = None):
    """Prints difference between start and end time.

    Args:
        start (float): Start time of computation (preferred in timeit format). 
        end (float): End time of computation.
        device ([type], optional): Device that compute is running on. Defaults to None.

    Returns:
        float: time between start and end in seconds (higher is longer).
    """
    total_time = end - start
    print(f"Train time on {device}: {total_time:.3f} seconds")
    return total_time

In [299]:
# Import tqdm for progress bar
from tqdm.auto import tqdm
# Set the seed and start the timer
train_time_start_on_cpu = timer()

epochs = 5

for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n-------")
    ### Training
    train_loss = 0
    # Add a loop to loop through training batches
    for batch, (X, y) in enumerate(train_dataloader):
        model_0.train() 
        # 1. Forward pass
        y_pred = model_0(X)

        # 2. Calculate loss (per batch)
        loss = loss_fn(y_pred, y)
        train_loss += loss # accumulatively add up the loss per epoch 

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model_0.parameters(), 5)
        
        # 5. Optimizer step
        optimizer.step()
        
        if batch % 400 == 0:
            # print(train_loss, loss)
            print(f"Looked at {batch * len(X)}/{len(train_dataloader.dataset)} samples")
            print(loss, train_loss, y_pred[:5])

    # Divide total train loss by length of train dataloader (average loss per batch per epoch)
    train_loss /= len(train_dataloader)
    test_loss = 0
    model_0.eval()
    with torch.inference_mode():
        for X, y in test_dataloader:
            # 1. Forward pass
            test_pred = model_0(X)
           
            # 2. Calculate loss (accumatively)
            test_loss += loss_fn(test_pred, y) # accumulatively add up the loss per epoch

        
        # Calculations on test metrics need to happen inside torch.inference_mode()
        # Divide total test loss by length of test dataloader (per batch)
        test_loss /= len(test_dataloader)

        # Divide total accuracy by length of test dataloader (per batch)

    ## Print out what's happening
    print(f"\nTrain loss: {train_loss:.5f} | Test loss: {test_loss:.5f}")

# Calculate training time      
train_time_end_on_cpu = timer()
total_train_time_model_0 = print_train_time(start=train_time_start_on_cpu, 
                                           end=train_time_end_on_cpu,
                                           device=str(next(model_0.parameters()).device))

  0%|                                                     | 0/5 [00:00<?, ?it/s]

Epoch: 0
-------
Looked at 0/268026 samples
tensor(2.8877e+11, grad_fn=<MseLossBackward0>) tensor(2.8877e+11, grad_fn=<AddBackward0>) tensor([[ 0.0040],
        [ 0.0089],
        [ 0.0056],
        [ 0.0019],
        [-0.0026]], grad_fn=<SliceBackward0>)
Looked at 25600/268026 samples
tensor(1.9408e+11, grad_fn=<MseLossBackward0>) tensor(9.7782e+13, grad_fn=<AddBackward0>) tensor([[3008.8982],
        [3356.0273],
        [2501.3608],
        [2841.8584],
        [3031.6113]], grad_fn=<SliceBackward0>)
Looked at 51200/268026 samples
tensor(2.2613e+11, grad_fn=<MseLossBackward0>) tensor(1.9205e+14, grad_fn=<AddBackward0>) tensor([[22095.7949],
        [20419.7266],
        [22700.3691],
        [21937.7051],
        [22505.0859]], grad_fn=<SliceBackward0>)
Looked at 76800/268026 samples
tensor(1.5161e+11, grad_fn=<MseLossBackward0>) tensor(2.8217e+14, grad_fn=<AddBackward0>) tensor([[77227.1250],
        [84475.1016],
        [75719.0469],
        [85018.2188],
        [78839.7188]], g

 20%|█████████                                    | 1/5 [00:45<03:03, 45.77s/it]


Train loss: 147451396096.00000 | Test loss: 106063806464.00000
Epoch: 1
-------
Looked at 0/268026 samples
tensor(1.7398e+11, grad_fn=<MseLossBackward0>) tensor(1.7398e+11, grad_fn=<AddBackward0>) tensor([[458067.3125],
        [275884.6875],
        [352571.8438],
        [344799.1250],
        [412028.9062]], grad_fn=<SliceBackward0>)
Looked at 25600/268026 samples
tensor(4.0369e+10, grad_fn=<MseLossBackward0>) tensor(4.2016e+13, grad_fn=<AddBackward0>) tensor([[358421.0625],
        [385880.0938],
        [311426.2500],
        [410702.7812],
        [268939.5000]], grad_fn=<SliceBackward0>)
Looked at 51200/268026 samples
tensor(1.1924e+11, grad_fn=<MseLossBackward0>) tensor(7.4987e+13, grad_fn=<AddBackward0>) tensor([[444523.5938],
        [396547.5938],
        [419107.5938],
        [321589.3438],
        [313249.9688]], grad_fn=<SliceBackward0>)
Looked at 76800/268026 samples
tensor(6.0660e+10, grad_fn=<MseLossBackward0>) tensor(1.1444e+14, grad_fn=<AddBackward0>) tensor([[3527

 40%|██████████████████                           | 2/5 [01:26<02:07, 42.59s/it]

...


Train loss: 73887891456.00000 | Test loss: 69760270336.00000
Epoch: 3
-------
Looked at 0/268026 samples
tensor(5.9812e+10, grad_fn=<MseLossBackward0>) tensor(5.9812e+10, grad_fn=<AddBackward0>) tensor([[250340.7812],
        [265524.4062],
        [493265.9062],
        [290584.8438],
        [271380.6250]], grad_fn=<SliceBackward0>)
Looked at 25600/268026 samples
tensor(6.7118e+10, grad_fn=<MseLossBackward0>) tensor(3.0463e+13, grad_fn=<AddBackward0>) tensor([[481461.1562],
        [283481.9062],
        [314323.9062],
        [995748.8125],
        [195743.4062]], grad_fn=<SliceBackward0>)
Looked at 51200/268026 samples
tensor(4.6707e+10, grad_fn=<MseLossBackward0>) tensor(5.2291e+13, grad_fn=<AddBackward0>) tensor([[358248.7500],
        [428057.0312],
        [254908.2656],
        [472849.8438],
        [511533.9688]], grad_fn=<SliceBackward0>)
Looked at 76800/268026 samples
tensor(1.6485e+10, grad_fn=<MseLossBackward0>) tensor(7.9466e+13, grad_fn=<AddBackward0>) tensor([[811031

 80%|████████████████████████████████████         | 4/5 [02:45<00:40, 40.41s/it]


Train loss: 59735916544.00000 | Test loss: 60367044608.00000
Epoch: 4
-------
Looked at 0/268026 samples
tensor(7.0982e+10, grad_fn=<MseLossBackward0>) tensor(7.0982e+10, grad_fn=<AddBackward0>) tensor([[5.1486e+05],
        [2.4373e+05],
        [4.8790e+05],
        [2.8820e+05],
        [1.6787e+00]], grad_fn=<SliceBackward0>)
Looked at 25600/268026 samples
tensor(2.4360e+10, grad_fn=<MseLossBackward0>) tensor(2.7617e+13, grad_fn=<AddBackward0>) tensor([[454031.6562],
        [555307.8750],
        [271190.0312],
        [104670.4062],
        [491036.1562]], grad_fn=<SliceBackward0>)
Looked at 51200/268026 samples
tensor(1.5383e+10, grad_fn=<MseLossBackward0>) tensor(4.8252e+13, grad_fn=<AddBackward0>) tensor([[273717.3125],
        [457590.5312],
        [224085.9375],
        [140063.2031],
        [316277.8125]], grad_fn=<SliceBackward0>)
Looked at 76800/268026 samples
tensor(1.1748e+10, grad_fn=<MseLossBackward0>) tensor(7.0679e+13, grad_fn=<AddBackward0>) tensor([[340468.7188

100%|█████████████████████████████████████████████| 5/5 [03:22<00:00, 40.51s/it]


Train loss: 53269274624.00000 | Test loss: 56063520768.00000
Train time on cpu: 202.551 seconds


In [270]:
X, y = next(iter(train_dataloader))
X.shape, y.shape
model_0.train() 
y_pred = model_0(X)
y_pred[:5]

tensor([[0.0120],
        [0.0139],
        [0.0106],
        [0.0110],
        [0.0098]], grad_fn=<SliceBackward0>)

In [271]:
loss = loss_fn(y_pred, y)
loss

tensor(2.2344e+11, grad_fn=<MseLossBackward0>)

In [272]:
optimizer.zero_grad()

In [273]:
loss.backward()

In [274]:
optimizer.step()

In [275]:
train_loss = 0
train_loss += loss

In [276]:
train_loss

tensor(2.2344e+11, grad_fn=<AddBackward0>)

In [104]:
torch.manual_seed(42)
def eval_model(model: torch.nn.Module, 
               data_loader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module):
    """Returns a dictionary containing the results of model predicting on data_loader.

    Args:
        model (torch.nn.Module): A PyTorch model capable of making predictions on data_loader.
        data_loader (torch.utils.data.DataLoader): The target dataset to predict on.
        loss_fn (torch.nn.Module): The loss function of model.
        accuracy_fn: An accuracy function to compare the models predictions to the truth labels.

    Returns:
        (dict): Results of model making predictions on data_loader.
    """
    predictions = []
    actual = []
    loss, acc = 0, 0
    model.eval()
    with torch.inference_mode():
        for X, y in data_loader:
            # Make predictions with the model
            y_pred = model(X)
            predictions.append(y_pred)
            actual.append(y)
            
            # Accumulate the loss and accuracy values per batch
            loss += loss_fn(y_pred, y)
        
        # Scale loss and acc to find the average loss/acc per batch
        loss /= len(data_loader)        
    return {"model_name": model.__class__.__name__, # only works when model was created with a class
            "model_loss": loss.item()}, predictions, actual

# Calculate model 0 results on test dataset
model_0_results, predictions, actual = eval_model(model=model_0, data_loader=test_dataloader,
    loss_fn=loss_fn
)
model_0_results

{'model_name': 'RealEstatePredictor', 'model_loss': nan}

In [105]:
len(predictions)

2063

In [106]:
len(actual)

2063

In [107]:
len(X_test)

132014

In [108]:
actual_output = []
for raw in actual:
    actual_output.extend(raw.flatten().tolist())
predictions_output = [] 
for raw in predictions:
    predictions_output.extend(raw.flatten().tolist())

In [109]:
df_actual = pd.DataFrame(actual_output,columns=['actual'])
df_predictions = pd.DataFrame(predictions_output,columns=['predictions'])

df_out = pd.concat([df_actual,df_predictions],axis=1)

In [110]:
df_out

,actual,predictions
0,419000.0,NaN
1,291000.0,NaN
2,103000.0,NaN
3,411690.0,NaN
4,459900.0,NaN
...,...,...
132009,275000.0,NaN
132010,494000.0,NaN
132011,900000.0,NaN
132012,175000.0,NaN
